In [58]:
import pandas as pd
import numpy as np
import jieba 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

获取数据

In [59]:
data = pd.read_csv('书籍评价.csv', encoding='gbk')
data

,Unnamed: 0,内容,评价
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评
7,7,破书一本,差评
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评
9,9,基础知识写的挺好的！,好评


数据基本处理

In [60]:
content = data['内容']
content

0                从编程小白的角度看，入门极佳。
1              很好的入门书，简洁全面，适合小白。
2      讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
3            前半部分讲概念深入浅出，要言不烦，很赞
4               看了一遍还是不会写，有个概念而已
5           中规中矩的教科书，零基础的看了依旧看不懂
6       内容太浅显，个人认为不适合有其它语言编程基础的人
7                           破书一本
8     适合完完全全的小白读，有其他语言经验的可以去看别的书
9                     基础知识写的挺好的！
10                           太基础
11            略_嗦。。适合完全没有编程经验的小白
12                      真的真的不建议买
Name: 内容, dtype: object

In [61]:
data.loc[data.loc[:, '评价'] == '好评', '评论编号'] = 1
data.loc[data.loc[:, '评价'] == '差评', '评论编号'] = 0
data

,Unnamed: 0,内容,评价,评论编号
0,0,从编程小白的角度看，入门极佳。,好评,1.0
1,1,很好的入门书，简洁全面，适合小白。,好评,1.0
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评,1.0
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评,1.0
4,4,看了一遍还是不会写，有个概念而已,差评,0.0
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评,0.0
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评,0.0
7,7,破书一本,差评,0.0
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评,0.0
9,9,基础知识写的挺好的！,好评,1.0


In [62]:
#选择停用词
stopwords = []
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    #print(lines)
    for tmp in lines:
        line = tmp.strip()
        # print(line)
        stopwords.append(line)

stopwords = list(set(stopwords))
stopwords

['',
 '`',
 '每每',
 '犹且',
 '大凡',
 '倘',
 '而又',
 '喔唷',
 '何乐而不为',
 '、',
 '从头',
 'μ',
 '就',
 '慢说',
 '除非',
 '因着',
 '几乎',
 '不要',
 '乘势',
 '则甚',
 '切不可',
 '莫不',
 '$',
 'ｅ］',
 '大致',
 '它',
 '许多',
 '粗',
 '方',
 '具体来说',
 '呜呼',
 '!',
 '尽可能',
 '不止',
 '曾经',
 '距',
 '然後',
 '6',
 '［①⑦］',
 '即',
 '［①ｆ］',
 '看样子',
 '毋宁',
 '不外乎',
 '://',
 '另外',
 '共同',
 '是',
 '［⑨］',
 '出',
 '▲',
 '巩固',
 '５：０',
 '嘎登',
 '与其',
 '从未',
 '陡然',
 '［②ｉ］',
 '再者',
 '大批',
 '趁着',
 '借',
 '除了',
 '以至',
 '［④ｂ］',
 '惟其',
 '简而言之',
 '看上去',
 '■',
 '临到',
 '之前',
 '独自',
 '准备',
 '略为',
 '＝［',
 '有著',
 '矣',
 '设使',
 '如上',
 '乌乎',
 '放量',
 '｛',
 '交口',
 '归根结底',
 '趁势',
 '［⑦］',
 '有点',
 '犹自',
 '么',
 '愿意',
 '谨',
 '当场',
 '累次',
 '另一方面',
 '［②ｆ］',
 '哪边',
 '因而',
 '哎',
 '［③ｃ］',
 '［①Ａ］',
 '反过来说',
 '尽管',
 '照',
 '管',
 '联袂',
 '嘻',
 '不变',
 '为了',
 '［③］',
 '其中',
 '来看',
 '才能',
 '》），',
 '日益',
 '後来',
 '任凭',
 '挨次',
 '有的是',
 '除此之外',
 '这么些',
 '梆',
 '继后',
 '顷刻',
 '该',
 '巴巴',
 '或',
 '乃至',
 '［⑤ｂ］',
 '没奈何',
 '一面',
 '来得及',
 '虽然',
 '尽',
 '由是',
 ';',
 '１．',
 '［①⑥］',
 '充其量',
 '︿',
 '(',
 '他人',

In [63]:
#内容处理
comment_list = []

for tmp in content:
    seg_list = jieba.cut(tmp, cut_all=False)
    seg_str = ','.join(seg_list)
    comment_list.append(seg_str)

In [64]:
#统计词个数
con =  CountVectorizer(stop_words=stopwords)
x = con.fit_transform(comment_list)
x.toarray()

c:\Users\18873\.conda\envs\d2l\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0

In [65]:
con.get_feature_names_out()

array(['一本', '一遍', '三个', '中规中矩', '依旧', '入门', '内容', '分讲', '前半部', '受益匪浅',
       '基础', '基础知识', '完完全全', '小白', '小白读', '建议', '很赞', '教科书', '有个', '极佳',
       '概念', '浅显', '深入浅出', '看不懂', '真的', '破书', '简洁', '细节', '经验', '编程',
       '要言不烦', '角度看', '讲解', '语言', '适合', '项目', '顾及'], dtype=object)

准备训练集和测试集

In [66]:
x_train = x.toarray()[:10, :]
y_train = data['评价'][:10]

x_test = x.toarray()[10:, :]
y_test = data["评价"][10:]

模型训练

In [67]:
mb = MultinomialNB(alpha=1)
mb.fit(x_train, y_train)
y_pre = mb.predict(x_test)
y_pre

array(['差评', '差评', '差评'], dtype='<U2')

In [68]:
y_test

10    差评
11    差评
12    差评
Name: 评价, dtype: object